## Simple SIM recon 

This notebook is used to call simrecon on a single image and PSF pair without directory parsing.

## 1.  Define code paths
Currently we hard code these and they need to be modified to run on different machines.  In the future we may move to a more intelligent approach like always having code exist beside the notebooks and using relative imports.  

In [14]:
%pylab inline
%load_ext autoreload
%autoreload 

import tifffile as tif
import os
import sys
import numpy as np

computer = 'bnort'

if computer == 'default':
    sys.path.insert(1, 'Y:\Cryo_data2\Data Processing Notebooks')
    sys.path.insert(1, 'Y:\Cryo_data2\Data Processing Notebooks\Scripts')
elif computer == 'bnort':
    sys.path.insert(1, r'C:\Users\bnort\work\Janelia\code\\simrecon\scripts\Scripts')
    sys.path.insert(1, r'C:\Users\bnort\work\Janelia\code\\simrecon\scripts')
else:
    pass

from simrecon_utils import simrecon

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
input_name = r'D:\Janelia\Data For Brian\Cryo-SIM Scope #2 Data (James System)\Not Anti-reflection coated\Successful samples\CELL 4 - 1.0W 300ms updated HWPQWP\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc'
input_name = r'D:\Janelia\Data 2024-06-30\488cm cell 5 good signal_20240627_131236  Fail\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc'
#input_name = r'D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian\CELL 4 - 1.0W 300ms updated HWPQWP pos_20240503_115909\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc'
#otf_name = r'D:\Janelia\Data 2024-06-03\PSF-OTF used (Davids set of 4 wavelengths)\201909_19-20_best\488 OTF Bead 8_20190919_141256.mrc'
#otf_name = r'C:\Users\bnort\work\Janelia\ims\computed_OTF_folder\488nmLin403OTF0.mrc'
#otf_name = r'C:\Users\bnort\work\Janelia\ims\computed_OTF_folder\488nmLinOTF0.mrc'
#otf_name = r'C:\Users\bnort\work\Janelia\ims\computed_OTF_folder\488nmLinOTF0bn_oldroi_mask.mrc'
#otf_name = r'C:\Users\bnort\work\Janelia\ims\computed_OTF_folder\488nmLinOTF0_legacy.mrc'
#otf_name = r'C:\Users\bnort\work\Janelia\ims\computed_OTF_folder\488nmLinOTF0_nomask2.mrc'
#wl = 488

input_name = r'D:\Janelia\Data 2024-08-05\561nm cell 5 good signal_20240627_151300\560 nm 615 45 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc'
otf_name = r'D:\Janelia\Data 2024-08-05\BEST PSFs\560 201909_19-20_best.mrc'
wl = 560

#input_name = r'D:\Janelia\Data 2024-06-30\488cm cell 5 good signal_20240627_131236  Fail sanity check\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc'
#otf_name = r'C:\Users\bnort\work\Janelia\ims\computed_OTF_folder\488nmLinOTF0_mask.mrc'
#wl = 488

print(otf_name)
print(input_name)

D:\Janelia\Data 2024-08-05\BEST PSFs\560 201909_19-20_best.mrc
D:\Janelia\Data 2024-08-05\561nm cell 5 good signal_20240627_151300\560 nm 615 45 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc


## Set default params

Here we set the default params that will be used if none of them are overwritten.


In [38]:

base_kwargs = dict(
                    nphases=5,
                    ndirs=3,
                    angle0= 1.29,
                    negDangle=True,              
                    na= 0.85,
                    nimm= 1.0,
                    zoomfact= 2.0, 
                    background= 100.0,           
                    wiener= 0.001,
                    fastSIM=True,
                    otfRA= True,
                    dampenOrder0=True,
                    k0searchall=True,
                    equalizez=True,
                    preciseapo=True,
                    gammaApo=0.7,
                    suppressR=15.0,
                    nthreads = 8,
                    makemodel = False
                )

In [43]:
#to_process = []
gammaApo = 0.3
suppressR = 10.0
wiener = 0.001
driftfix = True
nosuppress = True
makemodel = False
zpadto = 0 

user_text = 'gApo_'+str(gammaApo)+'_supR_'+str(suppressR)+'_w_'+str(wiener)+'_wl_'+str(wl)

if zpadto > 0:
    user_text += '_zpadto_'+str(zpadto)

if driftfix:
    user_text += '_driftfix'

if nosuppress:
    user_text += '_nosuppress'

if makemodel:
    user_text += '_model'
print(user_text)

gApo_0.3_supR_10.0_w_0.001_wl_560_driftfix_nosuppress


In [44]:
base_kwargs.update(dict(gammaApo=gammaApo, suppressR=suppressR, wiener=wiener, zpadto=zpadto, driftfix=driftfix, nosuppress=nosuppress, makemodel=makemodel))   # default Full frame Recon. parameters
    
sim_kwargs = dict(                                                                                                            
input_file= input_name,
otf_file= otf_name,
ls= (wl/1000)/2/0.81)
        
sim_kwargs.update(base_kwargs)
            
#create processed file output name
output_file = sim_kwargs["input_file"].replace(".mrc", '_proc'+'_' + user_text + ".mrc")
sim_kwargs["output_file"] = output_file
    
sim_output = simrecon(**sim_kwargs)
print(sim_output)
for i in range(len(sim_output)):
    sim_output[i] = sim_output[i].replace('\r', '')
with open(output_file.replace(".mrc", ".txt"), "w") as myfile:
    myfile.write(str(sim_kwargs))
    myfile.write("\n".join(sim_output))            

['nphases=5, ndirs=3\r', 'nx=1024, ny=1024, nz=16, nwaves=1, ntimes=1\r', 'nzotf=65, dkzotf=0.061538, nxotf=65, nyotf=1, dkrotf=0.060096\r', 'Number of threads used: 8\r', 'Creating FFT plans...\r', 'Separation matrix:\r', '  1.00000   1.00000   1.00000   1.00000   1.00000 \r', '  1.00000   0.30902  -0.80902  -0.80902   0.30902 \r', '  0.00000   0.95106   0.58779  -0.58779  -0.95106 \r', '  1.00000  -0.80902   0.30902   0.30902  -0.80902 \r', '  0.00000   0.58779  -0.95106   0.95106  -0.58779 \r', '\r', 'rdistcutoff = 2.905983 1/micron\r', 'Drift for direction 1 is (0.000000, 0.000000, 0.000000) pixels\r', 'Drift for direction 2 is (0.000000, 0.000000, 0.000000) pixels\r', 'k0guess[direction 0] = (0.400836, 1.389779) 1/micron\r', 'Initial guess by findk0() of k0[direction 0] = (0.289159,1.351632) 1/micron\r', 'before fitk0andmodamp\r', ' In getmodamp: angle=1.360040, mag=1.382216, amp=1.138258, phase=-2.010472\r', ' In getmodamp: angle=1.361040, mag=1.382216, amp=1.031778, phase=-1.654

In [22]:
print(sim_output)

['nphases=5, ndirs=3', 'nx=1024, ny=1024, nz=24, nwaves=1, ntimes=1', 'nzotf=65, dkzotf=0.061538, nxotf=81, nyotf=1, dkrotf=0.048077', 'Number of threads used: 8', 'Creating FFT plans...', 'Separation matrix:', '  1.00000   1.00000   1.00000   1.00000   1.00000 ', '  1.00000   0.30902  -0.80902  -0.80902   0.30902 ', '  0.00000   0.95106   0.58779  -0.58779  -0.95106 ', '  1.00000  -0.80902   0.30902   0.30902  -0.80902 ', '  0.00000   0.58779  -0.95106   0.95106  -0.58779 ', '', 'rdistcutoff = 3.269231 1/micron', 'k0guess[direction 0] = (0.459975, 1.594829) 1/micron', 'Initial guess by findk0() of k0[direction 0] = (0.457362,1.595617) 1/micron', 'before fitk0andmodamp', ' In getmodamp: angle=1.291645, mag=1.659872, amp=0.996309, phase=0.000003', ' In getmodamp: angle=1.292645, mag=1.659872, amp=0.996707, phase=0.000001', ' In getmodamp: angle=1.293645, mag=1.659872, amp=0.995770, phase=-0.000001', ' In getmodamp: angle=1.292443, mag=1.659872, amp=0.996725, phase=0.000001', ' In getmod